# Convolutional Neural Network

### Importing the libraries

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
import os

## Part 1 - Data Preprocessing

In [19]:
# Initialize the ImageDataGenerator for training set with augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Initialize the ImageDataGenerator for test set
validation_datagen = ImageDataGenerator(rescale=1.0/255)

### Preprocessing the Training set

In [21]:
training_set = train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [23]:
test_set = validation_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [25]:
cnn = Sequential()

### Step 1 - Convolution

In [27]:
# Add input layer explicitly
cnn.add(Input(shape=(64, 64, 3)))

# Add convolutional layer
cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu'))

### Step 2 - Pooling

In [29]:
cnn.add(MaxPooling2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [31]:
cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [33]:
cnn.add(Flatten())

### Step 4 - Full Connection

In [35]:
cnn.add(Dense(units=128, activation='relu'))
cnn.add(Dropout(0.5))  # Optional: Dropout to prevent overfitting

### Step 5 - Output Layer

In [37]:
cnn.add(Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [39]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [49]:
cnn.fit(
    x=training_set,
    validation_data=test_set,
    epochs=25
)

Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 28s 111ms/step - accuracy: 0.8420 - loss: 0.3399 - val_accuracy: 0.8015 - val_loss: 0.4650
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 25s 100ms/step - accuracy: 0.8419 - loss: 0.3455 - val_accuracy: 0.8175 - val_loss: 0.4676
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 25s 100ms/step - accuracy: 0.8611 - loss: 0.3208 - val_accuracy: 0.8105 - val_loss: 0.4768
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 25s 100ms/step - accuracy: 0.8523 - loss: 0.3297 - val_accuracy: 0.8210 - val_loss: 0.4312
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 26s 106ms/step - accuracy: 0.8653 - loss: 0.3093 - val_accuracy: 0.8035 - val_loss: 0.4580
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 28s 112ms/step - accuracy: 0.8635 - loss: 0.3137 - val_accuracy: 0.8115 - val_loss: 0.4648
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 27s 109ms/step - accuracy: 0.8727 - loss: 0.3019 - val_accuracy: 0.8185 - val_loss: 0.4511
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 27s 108ms/step - accuracy: 0.8707 - loss: 0

## Part 4 - Making a single prediction

In [43]:
def classify_single_image(image_path):
    test_image = image.load_img(image_path, target_size=(64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = cnn.predict(test_image)
    if training_set.class_indices['cats'] == 0:
        label = 'dog' if result[0][0] > 0.5 else 'cat'
    else:
        label = 'cat' if result[0][0] > 0.5 else 'dog'
    return label

In [45]:
# Predict images in "single_prediction" folder
single_prediction_dir = 'dataset/single_prediction'
for filename in os.listdir(single_prediction_dir):
    if filename.endswith(('.jpg', '.png')):
        file_path = os.path.join(single_prediction_dir, filename)
        prediction = classify_single_image(file_path)
        print(f'{filename}: {prediction}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
cat_or_dog_1.jpg: dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
cat_or_dog_2.jpg: cat
